In [1]:
"""
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""

'\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session\n'

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

path="others/"
initial_lgb_submission = pd.read_csv(path+'initial_lgb_submission.csv')
initial_lgb_submission

,ID,item_cnt_month
0,0,0.536405
1,1,2.151394
2,2,0.913563
3,3,0.721768
4,4,5.372386
...,...,...
214195,214195,0.125985
214196,214196,0.032229
214197,214197,0.074373
214198,214198,0.018177


In [2]:
blended_submission1 = pd.read_csv(path+'blended_submission1.csv')
blended_submission1

,ID,item_cnt_month
0,0,0.710527
1,1,0.346980
2,2,1.127346
3,3,0.457653
4,4,3.548673
...,...,...
214195,214195,0.104466
214196,214196,0.015815
214197,214197,0.050054
214198,214198,0.009008


In [3]:
before_submission = pd.read_csv('submission-0209-1900-simple-model.csv')
before_submission

,ID,item_cnt_month
0,0,1.06
1,1,1.55
2,2,1.55
3,3,1.55
4,4,1.55
...,...,...
214195,214195,1.65
214196,214196,1.65
214197,214197,1.65
214198,214198,1.65


# 데이터 로드 및 훈련, 테스트 데이터 확인

In [4]:
# Basic data loading
path = "data/"

df_train = pd.read_csv(path + 'sales_train.csv')
df_test = pd.read_csv(path + 'test.csv', index_col='ID')
df_shops = pd.read_csv(path + 'shops.csv', index_col='shop_id')
df_items = pd.read_csv(path + 'items.csv', index_col='item_id')
df_itemcat = pd.read_csv(path + 'item_categories.csv', index_col='item_category_id')
sample_submission = pd.read_csv(path + 'sample_submission.csv', index_col='ID')

In [5]:
df_train.info()
df_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [6]:
df_test

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
...,...,...
214195,45,18454
214196,45,16188
214197,45,15757


In [7]:
df_shops.head(10)

,shop_name
shop_id,
0,"!Якутск Орджоникидзе, 56 фран"
1,"!Якутск ТЦ ""Центральный"" фран"
2,"Адыгея ТЦ ""Мега"""
3,"Балашиха ТРК ""Октябрь-Киномир"""
4,"Волжский ТЦ ""Волга Молл"""
5,"Вологда ТРЦ ""Мармелад"""
6,"Воронеж (Плехановская, 13)"
7,"Воронеж ТРЦ ""Максимир"""
8,"Воронеж ТРЦ Сити-Парк ""Град"""


In [8]:
df_items.head(10)

,item_name,item_category_id
item_id,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40
1,!ABBYY FineReader 12 Professional Edition Full...,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40
4,***КОРОБКА (СТЕКЛО) D,40
5,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,40
6,***УДАР ПО ВОРОТАМ (UNI) D,40
7,***УДАР ПО ВОРОТАМ-2 (UNI) D,40
8,***ЧАЙ С МУССОЛИНИ D,40


In [9]:
df_itemcat.values[:20]

array([['PC - Гарнитуры/Наушники'],
       ['Аксессуары - PS2'],
       ['Аксессуары - PS3'],
       ['Аксессуары - PS4'],
       ['Аксессуары - PSP'],
       ['Аксессуары - PSVita'],
       ['Аксессуары - XBOX 360'],
       ['Аксессуары - XBOX ONE'],
       ['Билеты (Цифра)'],
       ['Доставка товара'],
       ['Игровые консоли - PS2'],
       ['Игровые консоли - PS3'],
       ['Игровые консоли - PS4'],
       ['Игровые консоли - PSP'],
       ['Игровые консоли - PSVita'],
       ['Игровые консоли - XBOX 360'],
       ['Игровые консоли - XBOX ONE'],
       ['Игровые консоли - Прочие'],
       ['Игры - PS2'],
       ['Игры - PS3']], dtype=object)

In [10]:
# https://www.kaggle.com/kazimanil/predict-future-sales-supplementary


path = "eng/"

df_eng_calendar = pd.read_csv(path + 'calendar.csv')
df_eng_shops = pd.read_csv(path + 'shops-translated.csv', index_col='shop_id')
df_eng_usd_rub = pd.read_csv(path + 'usd-rub.csv')
df_eng_itemcat = pd.read_csv(path + 'item_category.csv')

In [11]:
df_eng_calendar

,date,holiday,weekend
0,2013-01-01,1,0
1,2013-01-02,1,0
2,2013-01-03,1,0
3,2013-01-04,1,0
4,2013-01-05,0,1
...,...,...,...
1090,2015-12-27,0,1
1091,2015-12-28,0,0
1092,2015-12-29,0,0
1093,2015-12-30,0,0


In [12]:
df_eng_shops

,City,Type,Name
shop_id,,,
0,Yakutsk,Shop,"Ordzhonikidze, 56 francs"
1,Yakutsk,TC,Central franc
2,Adygea,TC,Mega
3,Balashikha,TRC,October-Kinomir
4,Volzhsky,TC,Volga Mall
5,Vologda,TRC,Marmalade
6,Voronezh,Shop,"(Plekhanovskaya, 13)"
7,Voronezh,TRC,Maksimir
8,Voronezh,TRC,"City-Park Grad"""


In [13]:
df_eng_itemcat

,item_id,item_name_translated,item_cat1,item_cat2
0,5441,PC: Headset HyperX Cloud Core gaming stereo (K...,PC,Headsets / Headphones
1,16255,Headphones PHILIPS SBC HC8680,PC,Headsets / Headphones
2,16256,Headphones RITMIX RH-120,PC,Headsets / Headphones
3,16257,Headphones RITMIX RH-124 Black,PC,Headsets / Headphones
4,5606,PS2: Memory Card 8 MB Black (Memory Card 8Mb -...,Accessories,PS2
...,...,...,...,...
22165,22095,Power supply GP 15AU (LR6) - Blister 2 pcs Ultra,Elements of a food,NaN
22166,22096,Power supply GP 24A (LR03) - Blister 2 pcs,Elements of a food,NaN
22167,22097,Power supply GP 24AU (LR03) - Blister 2 pcs Ultra,Elements of a food,NaN
22168,22098,Power supply COSMOS LR03 2 * BL,Elements of a food,NaN


In [14]:
df_train.shop_id.unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
        0,  1, 16, 15,  8, 10, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45,  5, 57, 58, 55, 17,  9,
       49, 39, 40, 48, 34, 33, 20, 11, 36])

In [15]:
df_test.shop_id.unique()


array([ 5,  4,  6,  3,  2,  7, 10, 12, 28, 31, 26, 25, 22, 24, 21, 15, 16,
       18, 14, 19, 42, 50, 49, 53, 52, 47, 48, 57, 58, 59, 55, 56, 36, 37,
       35, 38, 34, 46, 41, 44, 39, 45])

# 전처리
## 1. 테스트 데이터와 관련 된 shop_id, item_id 일치하는 훈련 데이터 추출

In [16]:
def check_unique_shop_item(df, name="train"):
    print("####",name,"shop_id unique ####")
    print(df["shop_id"].unique().shape)
    print(df["shop_id"].unique())

    print("\n####",name,"item_id unique ####")
    print(df["item_id"].unique().shape)
    print(df["item_id"].unique())
    print("-----------------\n")

In [17]:
print("#########train에서 test와 id가 동일한 것들만 추출하기전########")
check_unique_shop_item(df_train, name="train")
check_unique_shop_item(df_test, name="test")

#########train에서 test와 id가 동일한 것들만 추출하기전########
#### train shop_id unique ####
(60,)
[59 25 24 23 19 22 18 21 28 27 29 26  4  6  2  3  7  0  1 16 15  8 10 14
 13 12 53 31 30 32 35 56 54 47 50 42 43 52 51 41 38 44 37 46 45  5 57 58
 55 17  9 49 39 40 48 34 33 20 11 36]

#### train item_id unique ####
(21807,)
[22154  2552  2554 ...  7610  7635  7640]
-----------------

#### test shop_id unique ####
(42,)
[ 5  4  6  3  2  7 10 12 28 31 26 25 22 24 21 15 16 18 14 19 42 50 49 53
 52 47 48 57 58 59 55 56 36 37 35 38 34 46 41 44 39 45]

#### test item_id unique ####
(5100,)
[ 5037  5320  5233 ... 15757 19648   969]
-----------------



In [18]:
def extract_from_test_unique(train, test):
    # train데이터 중 test데이터에 있는 shop_id만 남깁니다.
    test_shops = test.shop_id.unique()
    train = train[train.shop_id.isin(test_shops)]
    print("shop_id 전처리 후 :",train.shape) # 2935849 -> 2413246

    # train데이터 중 test데이터에 있는 item_id만 남깁니다.
    test_items = test.item_id.unique()
    train = train[train.item_id.isin(test_items)]
    print("item_id 전처리 후 :",train.shape) # 2413246 -> 1224439
    return train, test

In [19]:
#df_train, df_test = extract_from_test_unique(df_train, df_test)
print("#########train에서 test와 id가 동일한 것들만 추출한 후########")
check_unique_shop_item(df_train, name="train")
check_unique_shop_item(df_test, name="test")

#########train에서 test와 id가 동일한 것들만 추출한 후########
#### train shop_id unique ####
(60,)
[59 25 24 23 19 22 18 21 28 27 29 26  4  6  2  3  7  0  1 16 15  8 10 14
 13 12 53 31 30 32 35 56 54 47 50 42 43 52 51 41 38 44 37 46 45  5 57 58
 55 17  9 49 39 40 48 34 33 20 11 36]

#### train item_id unique ####
(21807,)
[22154  2552  2554 ...  7610  7635  7640]
-----------------

#### test shop_id unique ####
(42,)
[ 5  4  6  3  2  7 10 12 28 31 26 25 22 24 21 15 16 18 14 19 42 50 49 53
 52 47 48 57 58 59 55 56 36 37 35 38 34 46 41 44 39 45]

#### test item_id unique ####
(5100,)
[ 5037  5320  5233 ... 15757 19648   969]
-----------------



## 2. 휴일 여부 추가

In [20]:
"""
df_eng_calendar["date"] = df_eng_calendar["date"].apply(lambda x: '.'.join(x.split("-")[::-1]))
df_train_cal = df_train.set_index("date").join(df_eng_calendar.set_index("date")).reset_index()
df_train_cal
"""

'\ndf_eng_calendar["date"] = df_eng_calendar["date"].apply(lambda x: \'.\'.join(x.split("-")[::-1]))\ndf_train_cal = df_train.set_index("date").join(df_eng_calendar.set_index("date")).reset_index()\ndf_train_cal\n'

## 3. 가게 카테고리 추가

In [21]:
df_train_shop = df_train.set_index("shop_id").join(df_eng_shops).reset_index()
df_train_shop = df_train_shop.drop(columns=["Name"])

df_test_shop = df_test.set_index("shop_id").join(df_eng_shops).reset_index()
df_test_shop = df_test_shop.drop(columns=["Name"])
df_train_shop

,shop_id,date,date_block_num,item_id,item_price,item_cnt_day,City,Type
0,0,12.01.2013,0,11059,73.0,1.0,Yakutsk,Shop
1,0,22.01.2013,0,11059,73.0,1.0,Yakutsk,Shop
2,0,26.01.2013,0,11059,73.0,1.0,Yakutsk,Shop
3,0,12.01.2013,0,11347,170.0,1.0,Yakutsk,Shop
4,0,22.01.2013,0,11347,170.0,1.0,Yakutsk,Shop
...,...,...,...,...,...,...,...,...
2935844,59,10.10.2015,33,4962,298.0,1.0,Yaroslavl,SC
2935845,59,04.10.2015,33,4706,299.0,1.0,Yaroslavl,SC
2935846,59,11.10.2015,33,5002,2999.0,1.0,Yaroslavl,SC
2935847,59,15.10.2015,33,5233,1199.0,1.0,Yaroslavl,SC


## 4. 아이템 카테고리 추가

In [22]:
df_train_item = df_train_shop.set_index("item_id").join(df_eng_itemcat.set_index("item_id")).reset_index()
df_train_item = df_train_item.drop(columns="item_name_translated")

df_test_item = df_test_shop.set_index("item_id").join(df_eng_itemcat.set_index("item_id")).reset_index()
df_test_item = df_test_item.drop(columns="item_name_translated")
df_train_item

,item_id,shop_id,date,date_block_num,item_price,item_cnt_day,City,Type,item_cat1,item_cat2
0,0,54,01.09.2014,20,58.0,1.0,Khimki,TC,Cinema,DVD
1,1,55,04.04.2014,15,4490.0,1.0,Web,Web,Programs,Home and Office (Figure)
2,1,55,02.04.2014,15,4490.0,1.0,Web,Web,Programs,Home and Office (Figure)
3,1,55,06.07.2014,18,4490.0,1.0,Web,Web,Programs,Home and Office (Figure)
4,1,55,04.08.2014,19,4490.0,1.0,Web,Web,Programs,Home and Office (Figure)
...,...,...,...,...,...,...,...,...,...,...
2935844,22168,12,13.09.2013,8,799.0,1.0,Web,Web,Gifts,"Gadgets, robots, sports"
2935845,22168,16,17.02.2013,1,799.0,1.0,Moscow,TC,Gifts,"Gadgets, robots, sports"
2935846,22168,42,02.02.2013,1,799.0,1.0,SanktPetersburg,TC,Gifts,"Gadgets, robots, sports"
2935847,22168,43,08.03.2013,2,798.5,1.0,SanktPetersburg,TC,Gifts,"Gadgets, robots, sports"


In [31]:
df_train_shop

,shop_id,date,date_block_num,item_id,item_price,item_cnt_day,City,Type
0,0,12.01.2013,0,11059,73.0,1.0,Yakutsk,Shop
1,0,22.01.2013,0,11059,73.0,1.0,Yakutsk,Shop
2,0,26.01.2013,0,11059,73.0,1.0,Yakutsk,Shop
3,0,12.01.2013,0,11347,170.0,1.0,Yakutsk,Shop
4,0,22.01.2013,0,11347,170.0,1.0,Yakutsk,Shop
...,...,...,...,...,...,...,...,...
2935844,59,10.10.2015,33,4962,298.0,1.0,Yaroslavl,SC
2935845,59,04.10.2015,33,4706,299.0,1.0,Yaroslavl,SC
2935846,59,11.10.2015,33,5002,2999.0,1.0,Yaroslavl,SC
2935847,59,15.10.2015,33,5233,1199.0,1.0,Yaroslavl,SC


## 2. 카테고리형 변수 라벨 인코딩

In [9]:
from sklearn.preprocessing import LabelEncoder


def encoder_df(train, test):

    train_enc = train.copy()
    #train_enc["shop_id"] = train_enc["shop_id"].astype('category')
    #train_enc["item_id"] = train_enc["item_id"].astype('category')
    test_enc = test.copy()
    #test_enc["shop_id"] = test_enc["shop_id"].astype('category')
    #test_enc["item_id"] = test_enc["item_id"].astype('category')
    return train_enc, test_enc

    #train_enc["item_cat2"] = train_enc["item_cat2"].fillna("empty")
    #test_enc["item_cat2"] = train_enc["item_cat2"].fillna("empty")
    #return train_enc, test_enc, encoders
"""
    cols = train_enc.columns[-2:]
    encoders = [LabelEncoder() for i in range(2)]

    for i in range(2):
        column = str(cols[i])
        #print(column)
        #print(train_enc[column])
        fit_data = pd.concat([train_enc[column], test_enc[column]], ignore_index=True)
        #print(fit_data)
        encoders[i].fit(fit_data)
        #encoders[i].fit(test_enc[column])
        train_enc[column] = encoders[i].transform(train_enc[column])
        test_enc[column] = encoders[i].transform(test_enc[column])
"""        


'\n    cols = train_enc.columns[-2:]\n    encoders = [LabelEncoder() for i in range(2)]\n\n    for i in range(2):\n        column = str(cols[i])\n        #print(column)\n        #print(train_enc[column])\n        fit_data = pd.concat([train_enc[column], test_enc[column]], ignore_index=True)\n        #print(fit_data)\n        encoders[i].fit(fit_data)\n        #encoders[i].fit(test_enc[column])\n        train_enc[column] = encoders[i].transform(train_enc[column])\n        test_enc[column] = encoders[i].transform(test_enc[column])\n'

In [10]:
df_train_enc, df_test_enc = encoder_df(df_train, df_test)

In [11]:
df_train_enc

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [12]:
df_test_enc

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
...,...,...
214195,45,18454
214196,45,16188
214197,45,15757


In [41]:
encoders[-1].inverse_transform(df_test_enc["item_cat2"])

KeyError: 'item_cat2'

In [28]:
encoders[-1].inverse_transform(df_train_enc["item_cat2"])

array(['DVD', 'Home and Office (Figure)', 'Home and Office (Figure)', ...,
       'Gadgets, robots, sports', 'Gadgets, robots, sports', 'Souvenirs'],
      dtype=object)

In [29]:
del df_train, df_test, df_test_shop, df_train_shop, df_train_item, df_test_item

In [13]:
# 시험용으로 변수 수 줄이기
df_train_enc_drop = df_train_enc[:int(df_train_enc.shape[0]/10)]

In [14]:
df_train_enc_drop

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
293579,09.03.2013,2,8,3960,299.00,1.0
293580,24.03.2013,2,8,3955,1649.00,1.0
293581,24.03.2013,2,8,3921,299.00,1.0
293582,21.03.2013,2,8,3823,399.00,1.0


In [15]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def date_processing(df):
    df["date"] = df["date"].apply(lambda date_str : datetime.strptime(date_str, "%d.%m.%Y"))

    df["year"] = df["date"].apply(lambda date : date.year)
    df["month"] = df["date"].apply(lambda date : date.month)
    df["day"] = df["date"].apply(lambda date : date.day)
    df = df.drop(columns=["date"])
    df = df[["year", "month","day", "date_block_num", "shop_id", "item_id",
             "item_price", "item_cnt_day"]]
    return df


df_train_preprocessed = df_train_enc_drop
df_train_preprocessed = date_processing(df_train_preprocessed)
df_train_preprocessed.head()

/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/aiffel/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,year,month,day,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013,1,2,0,59,22154,999.00,1.0
1,2013,1,3,0,25,2552,899.00,1.0
2,2013,1,5,0,25,2552,899.00,-1.0
3,2013,1,6,0,25,2554,1709.05,1.0
4,2013,1,15,0,25,2555,1099.00,1.0


In [16]:
df_train_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293584 entries, 0 to 293583
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            293584 non-null  int64  
 1   month           293584 non-null  int64  
 2   day             293584 non-null  int64  
 3   date_block_num  293584 non-null  int64  
 4   shop_id         293584 non-null  int64  
 5   item_id         293584 non-null  int64  
 6   item_price      293584 non-null  float64
 7   item_cnt_day    293584 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.9 MB


In [17]:
df_train_preprocessed

,year,month,day,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013,1,2,0,59,22154,999.00,1.0
1,2013,1,3,0,25,2552,899.00,1.0
2,2013,1,5,0,25,2552,899.00,-1.0
3,2013,1,6,0,25,2554,1709.05,1.0
4,2013,1,15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...,...,...
293579,2013,3,9,2,8,3960,299.00,1.0
293580,2013,3,24,2,8,3955,1649.00,1.0
293581,2013,3,24,2,8,3921,299.00,1.0
293582,2013,3,21,2,8,3823,399.00,1.0


In [18]:
#df_train_preprocessed["item_cnt_month"] = df_train_preprocessed.groupby(["year", "month", "shop_id", "item_id"])["item_cnt_day"].sum().reset_index()["item_cnt_day"]

In [30]:
cols = ["year", "month","date_block_num", "shop_id", "item_id"]
df_train_month = df_train_preprocessed.groupby(cols)["item_cnt_day"].sum().reset_index()

In [31]:
df_train_month

,year,month,date_block_num,shop_id,item_id,item_cnt_day
0,2013,1,0,0,32,6.0
1,2013,1,0,0,33,3.0
2,2013,1,0,0,35,1.0
3,2013,1,0,0,43,1.0
4,2013,1,0,0,51,2.0
...,...,...,...,...,...,...
159901,2013,3,2,57,5948,2.0
159902,2013,3,2,57,5964,1.0
159903,2013,3,2,57,5998,1.0
159904,2013,3,2,57,6002,1.0


In [32]:
df_train_month = df_train_month.rename(columns={"item_cnt_day": "item_cnt_month"})
df_train_month

,year,month,date_block_num,shop_id,item_id,item_cnt_month
0,2013,1,0,0,32,6.0
1,2013,1,0,0,33,3.0
2,2013,1,0,0,35,1.0
3,2013,1,0,0,43,1.0
4,2013,1,0,0,51,2.0
...,...,...,...,...,...,...
159901,2013,3,2,57,5948,2.0
159902,2013,3,2,57,5964,1.0
159903,2013,3,2,57,5998,1.0
159904,2013,3,2,57,6002,1.0


In [33]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
random_state = 200

month_X = df_train_month.iloc[:,:5]
month_y = df_train_month.iloc[:,5]

def month_predict(model, month_X, month_y, kind="default", print_len=7):
    X_train, X_val, y_train, y_val = train_test_split(month_X, month_y, test_size=0.2, random_state=random_state)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    model_name = model.__class__.__name__
    print("#### " +model_name+" month prediction("+kind+") ####")
    print("y_train[:{}] : {}".format(print_len, np.round(y_train[:print_len].values,1)))
    print("y_pred_train[:{}] : {}".format(print_len, np.round(y_pred_train[:print_len],1)))
    print()
    y_pred_val = model.predict(X_val)
    print("y_val[:{}] : {}".format(print_len, np.round(y_val[:print_len].values,1)))
    print("y_pred_val[:{}] : {}".format(print_len, np.round(y_pred_val[:print_len],1)))
    rmse = np.sqrt(mean_squared_error(y_pred_val, y_val))
    print("rmse : {:.6f}".format(rmse))
    print("----------\n")
    return model

In [34]:
from sklearn.ensemble import RandomForestRegressor
month_regressor = RandomForestRegressor(random_state=random_state)
month_regressor = month_predict(month_regressor, month_X, month_y)

#### RandomForestRegressor month prediction(default) ####
y_train[:7] : [1. 1. 1. 3. 1. 2. 1.]
y_pred_train[:7] : [1.4 1.1 1.  3.9 1.9 1.7 1.1]

y_val[:7] : [4. 7. 1. 8. 2. 8. 3.]
y_pred_val[:7] : [2.7 4.7 2.1 3.6 1.1 5.6 1.7]
rmse : 2.942642
----------



In [36]:
df_test["year"] = 2015
df_test["month"] = 11
df_test["date_block_num"] = 34

df_test = df_test[["year", "month", "date_block_num", "shop_id", "item_id"]]
df_test

,year,month,date_block_num,shop_id,item_id
ID,,,,,
0,2015,11,34,5,5037
1,2015,11,34,5,5320
2,2015,11,34,5,5233
3,2015,11,34,5,5232
4,2015,11,34,5,5268
...,...,...,...,...,...
214195,2015,11,34,45,18454
214196,2015,11,34,45,16188
214197,2015,11,34,45,15757


In [37]:
df_test["item_cnt_month"] = month_regressor.predict(df_test)
df_test

,year,month,date_block_num,shop_id,item_id,item_cnt_month
ID,,,,,,
0,2015,11,34,5,5037,1.93
1,2015,11,34,5,5320,1.67
2,2015,11,34,5,5233,1.30
3,2015,11,34,5,5232,1.30
4,2015,11,34,5,5268,1.94
...,...,...,...,...,...,...
214195,2015,11,34,45,18454,1.11
214196,2015,11,34,45,16188,2.62
214197,2015,11,34,45,15757,1.01


In [38]:
sample_submission["item_cnt_month"] = df_test["item_cnt_month"]
sample_submission

,item_cnt_month
ID,
0,1.93
1,1.67
2,1.30
3,1.30
4,1.94
...,...
214195,1.11
214196,2.62
214197,1.01


# Simplest Model
- public score :  3.88075

In [39]:
sample_submission.to_csv("submission-0209-2210-simple-model.csv")